In [ ]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import os

In [ ]:
Herb_TCMsymptom = pd.DataFrame(
    columns=['TCM symptom id', 'TCM symptom name', 'Symptom pinyin name', 
             'Symptom locus', 'Symptom property', 'Type'])

for i in tqdm(range(703)):

    rrid = f"SMHB{i+1:05d}"

    try:
        data = pd.read_csv(f'../rawdata/Herb-TCMSymptom/{rrid}-TCMSymptom.csv')
        data['Herb'] = rrid

        if not Herb_TCMsymptom.empty:
            Herb_TCMsymptom = pd.concat([Herb_TCMsymptom, data], axis=0)
        else:
            Herb_TCMsymptom = data
    except:
        pass

Herb_TCMsymptom = Herb_TCMsymptom[['Herb', 'TCM symptom id']]
Herb_TCMsymptom.columns = ['Herb', 'TCMSymptom']
Herb_TCMsymptom['node1_type'] = 'Herb'
Herb_TCMsymptom['node2_type'] = 'TCMSymptom'
Herb_TCMsymptom = Herb_TCMsymptom.dropna().drop_duplicates()
Herb_TCMsymptom

In [ ]:
TCMSymptom_MMSymptom = pd.DataFrame(
    columns=['MM symptom id', 'MM symptom name', 'MeSH tree numbers', 'UMLS id'])

for i in tqdm(range(2364)):

    rrid = f"SMTS{i+1:05d}"

    try:
        data = pd.read_csv(f'../rawdata/TCMSymptom-MMSymptom/{rrid}-MMSymptom.csv')
        data['TCMSymptom'] = rrid

        if not TCMSymptom_MMSymptom.empty:
            TCMSymptom_MMSymptom = pd.concat([TCMSymptom_MMSymptom, data], axis=0)
        else:
            TCMSymptom_MMSymptom = data
    except:
        pass

TCMSymptom_MMSymptom = TCMSymptom_MMSymptom[['TCMSymptom', 'MM symptom id']]
TCMSymptom_MMSymptom.columns = ['TCMSymptom', 'MMSymptom']
TCMSymptom_MMSymptom['node1_type'] = 'TCMSymptom'
TCMSymptom_MMSymptom['node2_type'] = 'MMSymptom'
TCMSymptom_MMSymptom = TCMSymptom_MMSymptom.dropna().drop_duplicates()
TCMSymptom_MMSymptom

In [ ]:
MMSymptom_Disease = pd.DataFrame(
    columns=['Disease id', 'Disease name', 'OMIM id', 'Orphanet id'])

for i in tqdm(range(1148)):

    rrid = f"SMMS{i+1:05d}"

    try:
        data = pd.read_csv(f'../rawdata/MMSymptom-Disease/{rrid}-Disease.csv')
        data['MMSymptom'] = rrid

        if not MMSymptom_Disease.empty:
            MMSymptom_Disease = pd.concat([MMSymptom_Disease, data], axis=0)
        else:
            MMSymptom_Disease = data
    except:
        pass

MMSymptom_Disease = MMSymptom_Disease[['MMSymptom', 'Disease id']]
MMSymptom_Disease.columns = ['MMSymptom', 'Disease']
MMSymptom_Disease['node1_type'] = 'MMSymptom'
MMSymptom_Disease['node2_type'] = 'Disease'
MMSymptom_Disease = MMSymptom_Disease.dropna().drop_duplicates()
MMSymptom_Disease

In [ ]:
folder_path = '../rawdata/Disease-Target'

total_files = 14434

data_list = []

for i in tqdm(range(total_files)):
    rrid = f"SMDE{i+1:05d}"
    file_path = os.path.join(folder_path, f'{rrid}-Target.csv')
    try:
        data = pd.read_csv(
            file_path,
            usecols=['Gene symbol'],
            dtype={'Gene symbol': str},
            low_memory=False
        )
        data['Disease'] = rrid
        data = data[['Disease', 'Gene symbol']]
        data_list.append(data)
    except FileNotFoundError:
        continue
    except pd.errors.EmptyDataError:
        continue
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        continue

Disease_Target = pd.concat(data_list, ignore_index=True)
Disease_Target.columns = ['Disease', 'Target']
Disease_Target['node1_type'] = 'Disease'
Disease_Target['node2_type'] = 'Target'
Disease_Target = Disease_Target.dropna().drop_duplicates()

Disease_Target = Disease_Target[~Disease_Target['Target'].isin(['3.8-1.4', '3.8-1.5'])]

Disease_Target

In [ ]:
Herb_TCMsymptom.columns = ['node1', 'node2', 'node1_type', 'node2_type']
TCMSymptom_MMSymptom.columns = ['node1', 'node2', 'node1_type', 'node2_type']
MMSymptom_Disease.columns = ['node1', 'node2', 'node1_type', 'node2_type']
Disease_Target.columns = ['node1', 'node2', 'node1_type', 'node2_type']

In [ ]:
HTMDT = pd.concat([Herb_TCMsymptom, TCMSymptom_MMSymptom, MMSymptom_Disease, Disease_Target], axis = 0)

In [ ]:
HTMDT.to_csv('../preprocessed_data/HTMDT.csv', index=None)